In [1]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time
import re

BASE_URL = "https://github.com"

# ==============================
# DB（SQLite）作成（重複を避けるため name を UNIQUE に）
# ==============================
conn = sqlite3.connect("github_repos.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS repos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE,
    language TEXT,
    stars INTEGER
)
""")
conn.commit()


# ==============================
# 1ページ分をスクレイピング
# ==============================
def scrape_page(user, page):
    url = f"{BASE_URL}/{user}?page={page}&tab=repositories"
    print(f"[ページ取得中] {url}")

    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Failed to fetch page {page}: {resp.status_code}")
        return []

    soup = BeautifulSoup(resp.text, "html.parser")

    # GitHub の一般的なリポジトリ行セレクタ（複数パターンに対応）
    repo_items = soup.select("li.Box-row, div.Box-row, li.col-12")

    if not repo_items:
        return []

    repos = []

    for item in repo_items:
        # -----------------------
        # リポジトリ名
        # -----------------------
        name_tag = item.select_one("a[itemprop='name codeRepository'], h3 a, a[href*='/" + user + "/']")
        name = name_tag.get_text(strip=True) if name_tag else None

        if not name:
            # 取得できなければ次へ
            continue

        # GitHub の表示だと "username / reponame" みたいな場合があるので、スラッシュが入っていれば最後の部分を使う
        if "/" in name:
            name = name.split("/")[-1].strip()

        # -----------------------
        # 言語
        # -----------------------
        lang_tag = item.find("span", itemprop="programmingLanguage")
        language = lang_tag.get_text(strip=True) if lang_tag else None

        # -----------------------
        # スター数（stargazersリンク）
        # -----------------------
        star_tag = item.find("a", href=lambda href: href and "stargazers" in href)
        if star_tag:
            stars_text = star_tag.get_text(strip=True).replace(",", "")
            # 空文字や非数値のときは 0 にする
            try:
                stars = int(re.sub(r"[^\d]", "", stars_text)) if stars_text else 0
            except:
                stars = 0
        else:
            stars = 0

        repos.append({
            "name": name,
            "language": language,
            "stars": stars
        })

    return repos


# ==============================
# 全ページ取得（max_pages 引数で上限を設定）
# ==============================
def scrape_all(user, max_pages=6):
    all_repos = []
    page = 1

    while True:
        if page > max_pages:
            print(f"\n--- 最大ページ数 {max_pages} に到達したため停止します ---\n")
            break

        repos = scrape_page(user, page)

        if not repos:  # データが無くなった → 終了
            break

        all_repos.extend(repos)
        page += 1
        time.sleep(1)  # 1秒待つ（礼儀 & レート制限対策）

    return all_repos


# ==============================
# DB に保存（重複は置き換える）
# ==============================
def save_to_db(repos):
    for r in repos:
        # 名前がユニークなので既存は上書き（更新）する
        cur.execute(
            "INSERT OR REPLACE INTO repos (name, language, stars) VALUES (?, ?, ?)",
            (r["name"], r["language"], r["stars"])
        )
    conn.commit()


# ==============================
# DB から読み出し
# ==============================
def show_repos():
    cur.execute("SELECT id, name, language, stars FROM repos ORDER BY stars DESC")
    rows = cur.fetchall()

    print("\n==== 保存されたリポジトリ一覧 ====\n")
    for row in rows:
        print(f"ID:{row[0]}  名前:{row[1]}  言語:{row[2]}  ⭐:{row[3]}")


# ==============================
# メイン処理（呼び出し例）
# ==============================
if __name__ == "__main__":
    user = "google"          # ここを変更
    max_pages = 3            # 取得上限ページ数を指定（例：3ページ）

    repos = scrape_all(user, max_pages=max_pages)  # ← ここでエラーが出なくなる
    save_to_db(repos)
    show_repos()

    conn.close()


[ページ取得中] https://github.com/google?page=1&tab=repositories
[ページ取得中] https://github.com/google?page=2&tab=repositories
[ページ取得中] https://github.com/google?page=3&tab=repositories

--- 最大ページ数 3 に到達したため停止します ---


==== 保存されたリポジトリ一覧 ====

ID:13  名前:skia  言語:C++  ⭐:10274
ID:29  名前:skia  言語:C++  ⭐:10274
ID:45  名前:skia  言語:C++  ⭐:10274
ID:64  名前:osv-scanner  言語:Go  ⭐:8084
ID:80  名前:osv-scanner  言語:Go  ⭐:8084
ID:96  名前:osv-scanner  言語:Go  ⭐:8084
ID:10  名前:osv-scanner  言語:Go  ⭐:8083
ID:26  名前:osv-scanner  言語:Go  ⭐:8083
ID:42  名前:osv-scanner  言語:Go  ⭐:8083
ID:12  名前:perfetto  言語:C++  ⭐:5017
ID:28  名前:perfetto  言語:C++  ⭐:5017
ID:44  名前:perfetto  言語:C++  ⭐:5017
ID:59  名前:angle  言語:C++  ⭐:3843
ID:75  名前:angle  言語:C++  ⭐:3843
ID:91  名前:angle  言語:C++  ⭐:3843
ID:15  名前:brax  言語:Jupyter Notebook  ⭐:2941
ID:31  名前:brax  言語:Jupyter Notebook  ⭐:2941
ID:47  名前:brax  言語:Jupyter Notebook  ⭐:2941
ID:14  名前:boringssl  言語:C++  ⭐:2050
ID:30  名前:boringssl  言語:C++  ⭐:2050
ID:46  名前:boringssl  言語:C++  ⭐:2050
ID:9  名